Introduction

We are data analysts employeed by the NCAA. We were given the assigment to study the given dataset to find any noticeable trends that should be researched further. We are going to focus on the Academic Progress Rate (APR), which is a calculation based off how many athletes on a team remain academiclly elligible and with the university. Most of the data within the NCAA dataset is not gathered by the NCAA. In fact, almost all of it is gathered by the member schools, and then shared with the NCAA. However, some of this data has been directly gathered by the NCAA. We will be attempting to answer the following four questions:

1. How have the retention rates for NCAA schools changed over the available data, from 2004 to 2019? We will be using all of the RET_Rate_XXXX columns where XXXX represents a different year for each column

2. Exploratory analysis: How many sports are fielded by each institution, and does that vary between NCAA divisions?  We will be using the SCL_UNITID and SCL_NAME columns as well as the Sport_Code and Sport_Name to analyse the counts of sports by each university. This is important to see if schools generally field the same number of sports or if there's drastic differences between schools and analyze what could be different from schools with a high number of sports and those with a low number.

3. What variables are the most positively correlated with multi-year APR, and can we accurately predict what a teams multi-year APR will be given other data? (regression). This will involve using linear regression on the whole dataset to determine which variables are significant.

4. Do Private or Public schools have better APR and academic eligibility rates? We will be using the multi-year APR and multi-year academic eligibility columns and SCL_PRIVATE column.

To start with though, we need to clean the dataset.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm

In [ ]:
raw_data = pd.read_csv('C:/Users/natha/OneDrive/Documents/GitHub/DIDA_325_Final-project/csvs/raw data.csv')
drops=[0,12,20,85]
raw_data.drop(raw_data.columns[drops], axis=1, inplace=True)
# These columns were dropped because they were entirely empty.

cols_to_fill = list(range(82, 97)) # the columns we want to fill are the 83rd to 98th columns (0-indexed) so we are creating a list of their indices using the range function. 
raw_data.iloc[:, cols_to_fill] = raw_data.iloc[:, cols_to_fill].fillna(0)
# These columns were filled with 0s because some were empty but the documnetation said they should be 0s.

In [ ]:
# Next we will see what columns are still empty.
# We will do this by using the isnull() method to check for null values and then summing them up with the sum() method.

null_counts = raw_data.isnull().sum()
null_counts = null_counts[null_counts > 0]

print(null_counts.sort_values(ascending=False))

In [ ]:
#D1_FB_CONF_19 has a lot of null values, and is slightly redundant with CONFNAME_19 so we will drop it.
raw_data.drop(columns=['D1_FB_CONF_19'], inplace=True)
# Additionmally, any row with null values in the MULTIYR stats are useless to us so we will drop them as well.
clean_data=raw_data.dropna(subset=['MULTIYR_RET_RATE', 'MULTIYR_APR_RATE_1000_RAW', 'MULTIYR_APR_RATE_1000_CI', 'MULTIYR_APR_RATE_1000_OFFICIAL', 'MULTIYR_SQUAD_SIZE', 'MULTIYR_ELIG_RATE'])
# we use the subset paramter to specify which columns we want to check for null values and drop the rows with null values in those columns. https://www.w3schools.com/python/pandas/ref_df_dropna.asp

The columns remaining are SCL_UNITID (int) which is the id for each column, SCL_NAME (character), the name for the university, Sport_Code (Cateogry) and Sport_Name (chategory) which represent the id and name of the respective sport, Academic_Year(category) the year for that row, SCL_DIV_19 (Category), and SCL_SUB_19 (Category) which represent the NCAA division and subdivision they play in. CONFNAME (character) is the schools conference they compete in. SL_HBCU (category) and SL_Private (category) which is a binary data that shows if a school is HBCU and if a school is private. Then there are MULTIYR_APR_RATE_1000_RAW, MULTIYR_APR_RATE_1000_CI, MULTIYR_APR_RATE_1000_OFFICIAL, MULTIYR_SQUAD_SIZE, MULTIYR_ELIG_RATE,MULTIYR_RET_RATE which are all numeric and represent multiyear academic stats. Then from each year from 2006-2019 there are APR_RATE_XXXX_1000, ELIG_RATE_XXXX, RET_RATE_XXX, NUM_OF_ATHLETES_XXXX which are all numeric and represent single year academic stats. There are a lot of each column these are which each category of column means

APR_Rate: Academic Progress Rate as calculated by the NCAA.

ELIG_RATE: Percentage of athletes on a team which are academically elligible to compete.

RET_RATE: Percenentage of athletes who remain on the team after a year (excluding gratuating seniors and transfers)

NUM_OF_ATHLETES/MULTIYR_SQUAD_SIZE: The number of athletes on a sports teams roster.

Finally from 2006-2020 there are PUB_AWARD_XX which are categorical values that display if a school got an academic award from the NCAA.

In [ ]:
# This leaves us with the following dataset
clean_data.head()

Data Analaysis

Question 3: What variables are the most positively correlated with multi-year APR, and can we accurately predict what a teams multi-year APR will be given other data? (regression). This will involve using linear regression on the whole dataset to determine which variables are significant.

There are a couple of reasons why this is an important question to answer. One being that if the NCAA can predict what teams will have a higher or lower APR it knows which schools and sports it can provide athletes with more resources to help the students. Additionally if the NCAA knows which factors are most important for predicting APR they can know exactly how to help their athletes. Before answering the question I do beleive we'll be able to train a linear regression model that does a strong job at predicting APR, additionally I beleive multi year retention rates and multi year eligibility rates will be the two strongest variables in prediciting APR.


In [ ]:
data=clean_data.copy()
# Creating a copy of the dataset to work with this specific research qeustion
x_columns=['SCL_HBCU', 'SCL_PRIVATE', 'MULTIYR_SQUAD_SIZE', 'SCL_SUB_19', 'MULTIYR_ELIG_RATE', 'MULTIYR_RET_RATE'] # these are all the columns that will be used as predictive variables
X=data[x_columns] 
X = pd.get_dummies(X, drop_first=True).astype("float32") #For SCL_HCU and SCL_PRIVATE, we need to convert them to numeric values. We use get_dummies to convert them to binary variables. .
y=data['MULTIYR_APR_RATE_1000_OFFICIAL'] #This is our response variable, the APR for each team
X = sm.tools.tools.add_constant(X) #Add a constant to the model. This is necessary for the OLS regression model.
model=sm.OLS(y, X).fit() 
print(model.summary())

From the model summary we can see that "MULTIYR_SQUAD_SIZE", "MULTIYR_ELIG_RATE", "MULTIYR_RET_RATE", and "SCL_HCBU_Y" are all signifiicant variables as their p-values round to zero and using the coefficients we can gleam the following
MULTIYR_SQUAD_SIZE:
Every additional 1 person added to the squad size decreases the APR score by 0.0156 points, on average, holding other variables constant.
MULTIYR_ELIG_RATE:
Every 1.0 (or 100 percentage points) increase in eligibility rate would increase APR by 424.32 points. More realistically, a 1% increase (i.e., +0.01) in eligibility rate would increase APR by about 4.24 points.
MULTIYR_RET_RAT:
Every 1.0 (or 100 percentage points) increase in retention rate would increase APR by 420.22 points. So a 1% increase (i.e., +0.01) in retention rate would increase APR by about 4.20 points.
SCL_HBCU_Y:
If a school is an HBCU (coded as 1), then the APR is higher by about 1.92 points compared to non-HBCUs (coded as 0), holding everything else constant.

Additiionally with an R^2 of about 0.896 we can say that 89.6% of the variance of APR can be explained by SCL_HBCU', 'SCL_PRIVATE', 'MULTIYR_SQUAD_SIZE', 'SCL_SUB_19', 'MULTIYR_ELIG_RATE',  and 'MULTIYR_RET_RATE'. With the model we can predict APR scores decently accurately, compared to simply guessing the mean APR, and about 90% of those prediction improvement come from these six variables.

In [ ]:
yhat = model.predict()
residuals = y - yhat
ax = sns.scatterplot(x=yhat, y=residuals)
ax.axhline(y=0, color='black', linestyle='dashed')

plt.xlabel("Predicted APR")
plt.ylabel("Residual")
plt.title("Residuals vs Predicted APR")
plt.show()

There do not seem to be any trends with the residuals, which is a good thing for our model.